In [1]:
import json
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator, Vocab
from nltk.stem import PorterStemmer
from typing import List
from torch import nn
import torch
from einops import rearrange
import random
from tqdm import tqdm
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.corpus import wordnet
import re

In [2]:
wnl = WordNetLemmatizer()
tokenizer = get_tokenizer('spacy')
ps = PorterStemmer()

In [3]:
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

In [4]:
with open('docs.json') as f:
    contents = f.read()
    docs = json.loads(contents)

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gram = 9
bz = 256
epochs = 100
emb_dim = 128
dims = [256,]

In [6]:
def yield_tokens(docs):
    for doc in docs:
        if "abstract" in doc:
            for abstract in doc["abstract"]:
                if "text" in abstract:
                    # sen = re.sub('[^a-z]+', " ", abstract["text"].lower())
                    # sen = re.sub('[-]+', " ", abstract["text"].lower())
                    sen = abstract["text"].lower()
                    # yield [ps.stem(w) for w in tokenizer(sen)]
                    lemmas_sent = []
                    for tag in pos_tag(tokenizer(sen)):
                        wordnet_pos = get_wordnet_pos(tag[1]) or wordnet.NOUN
                        lemmas_sent.append(wnl.lemmatize(tag[0], pos=wordnet_pos)) 
                    yield lemmas_sent

vocab = build_vocab_from_iterator(yield_tokens(docs), specials=["<unk>"], min_freq=10)
vocab.set_default_index(vocab["<unk>"])

vocab.__len__()

5436

In [7]:
datas = []
for ws in yield_tokens(docs):
    for i in range(n_gram//2, len(ws) - n_gram//2):
        ctx = [*ws[i-n_gram//2:i], *ws[i+1:i+1+n_gram//2],]
        tar = ws[i]
        datas.append((ctx, tar))

In [8]:
class CBOW(torch.nn.Module):
    def __init__(self, n_gram, emb_dim, dims, vocab,):
        super(CBOW, self).__init__()
        self.vocab = vocab
        self.n_gram = n_gram
        
        self.emb_book = nn.Embedding(vocab.__len__(), emb_dim)
        
        # dims = [emb_dim*(n_gram-1), *dims]
        # dims = [emb_dim*2, *dims]
        dims = [emb_dim, *dims]
        net = []
        for i in range(len(dims)-1):
            net.append(nn.Linear(dims[i], dims[i+1]))
            net.append(nn.ReLU(),)
            
        net.append(nn.Linear(dims[-1], vocab.__len__()),)
        net.append(nn.LogSoftmax(dim = -1),)
            
        self.net = nn.ModuleList(net)

    def forward(self, ctx_idx):
        inp = self.emb_book(ctx_idx)
        out = inp.mean(1)
        # out = rearrange(inp, "b n d -> b (n d)")
        # f, b = torch.chunk(inp, 2, 1)
        # out = torch.cat([f.mean(1), b.mean(1)], 1)
        for layer in self.net:
            out = layer(out)
        
        return out

In [9]:
model = CBOW(n_gram, emb_dim, dims, vocab).to(device)
loss_fn = nn.NLLLoss()

optim = torch.optim.Adamax([*model.parameters(),], lr=0.01)

In [10]:
for epoch in range(epochs):
    total_loss = 0
    random.shuffle(datas)
    bnum = len(datas)//bz + (1 if len(datas)%bz!=0 else 0)
    for bidx in tqdm(range(bnum)):
        ctx = []
        tar = []
        for _ctx, _tar in datas[bidx*bz:(bidx+1)*bz]:
            ctx.append(vocab(_ctx))
            tar.append(_tar)
        
        ctx = torch.tensor(ctx, dtype=torch.int64, device=device)
        tar = torch.tensor(vocab(tar), dtype=torch.int64, device=device)
        
        out = model(ctx)
        loss = loss_fn(out, tar)
        optim.zero_grad()
        loss.backward()
        optim.step()
        
        total_loss += loss.item()*len(tar)
        
    print("epoch: {}, loss: {}".format(epoch, total_loss/len(datas)))
    torch.save({
        'epoch': epoch,
        'net_state_dict': model.state_dict(),
        'opt_state_dict': optim.state_dict(),
        }, "./cbow-ex7.pt")

100%|██████████| 5721/5721 [00:16<00:00, 353.06it/s]


epoch: 0, loss: 5.348953632015232


100%|██████████| 5721/5721 [00:17<00:00, 325.56it/s]


epoch: 1, loss: 4.814663781334909


100%|██████████| 5721/5721 [00:15<00:00, 359.93it/s]


...

epoch: 97, loss: 3.643131605773126


100%|██████████| 5721/5721 [00:16<00:00, 357.32it/s]


epoch: 98, loss: 3.6424955940593136


100%|██████████| 5721/5721 [00:17<00:00, 330.37it/s]


epoch: 99, loss: 3.6431357230354418


In [11]:
import io

out_v = io.open('vectors-ex7.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata-ex7.tsv', 'w', encoding='utf-8')

emb_book = model.emb_book.weight.detach().cpu().numpy()
for idx, w in enumerate(vocab.get_itos()):
    emb = emb_book[idx]
    out_v.write('\t'.join([str(x) for x in emb]) + "\n")
    out_m.write(w + "\n")
out_v.close()
out_m.close()